<a href="https://colab.research.google.com/github/vlx300/kb_colab/blob/master/HTTP_Requests_for_Humans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Python HTTP Request Library**
**REST**: Essentially a set of conventions for structuring a WEB API (an API you interect with via HTTP: making requests to specific URLS and gathering relevant data in the Response. You leverage the several HTTP methods GET, POST, UPDATE, DELETE etc. 


![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQK4tRxuYv7nnqsPCZWnzlXJgmNrUbldBIeNiBUsGA_2UWu72Sd&s)

In [0]:
import requests 

# Make an API call  and store the response #
url = 'https://api.github.com/search/repositories?q=language:python&sort=stars'
r = requests.get(url)
print("Status code:", r.status_code)

##**Web API Call alternative**

In [0]:
requests.get('https://api.github.com')

#**The Response**

A response is a powerful object for inspecting the results of a request. Make the same requet again and store the retun value in a variable so you can take a closer look at the data. Elegant

In [0]:
response = requests.get('https://api.github.com')

In this example, you've captured the "return value" of GET(), which is an instance of response, and store it in a variable called response. Now you can use response to see alot of information about the results of your GET request. 

##**Status Codes**

the first bit of information you can gather forma response is the status code. this informs you of the status of the request. For Exp. 200 means the request was successful . whereas  404 = NOT FOUND, 403 = NOT Authorized etc.. 302 = Redirect 

**Note**: *By accessing .status_code, you can see the status of the request returned from the Server.  Sometimes you might want to make decision in your code based upo the response "status code"* 

In [0]:
response.status_code # checks the status from the Server 

Making decisions (Other Logic) based on status code:

In [0]:
if response.status_code == 200: 
  print("Success")
elif response.status_code == 400:
  print("Not Found")
elif response.status_code == 302:
  print("Follow Redirect")
elif response.status_code == 302:
  print("Not Authorized")

If your response is used in a conditional expresssion, it will evaluate to **True** if that satus code is between 200 and 400 and **False** Otherwise. so you can simply the above ike this (See Below)..  

Note: *this is possible due to __Bool__() overload method on response.  Translation, the default behaviour of response has be modified to take status code into account when determining the "Truth value" of the object* so make sure to utilize ths convenient shorthand only if you want to know if a request was "generally sucessfull" 

In [0]:
if response:
  print("Success")
else:
  print('An error has occured')

Say if you dont want to check the status code like above but want to raise an exception if the request was unsuccessful only. You can using '.raise_for_status'

In [0]:
import requests
from requests import HTTPError

for url in ['http://api.github.com', 'https://api.github.com/invalid']:
  try:
    reponse = requests.get(url)

    # if the response was successful, no exception will be raised
    response.raise_for_status()
  except HTTPError as http_err:
    print(f'HTTP error has occured: {http_err}')  #python 3.6
  except Exception as err:
    print(f'Other error occured: {err}') #Python 3.6 
  else:
    print("success")


#**API Content**

The reponse of the HTTP API GET request has some valuable information i.e..Payload in the message body. Using the attributes of Response, you can view the payload in many different formats.

Bytes (.Content)

In [0]:
response = requests.get('https://api.github.com')
response.content  # give you the content in raw bytes of the response payload

In [0]:
response.text # convert into a string using encoding like UTF-8

Decoding bytes to a string requires an encoding scheme. request will try to guess encoding based upon the response headers if not specified.  Set encoding BEFORE accessing .text

In [0]:
response.encoding = 'utf-8'  #optional enciding parameter.  requests infers this
response.text

##**Serialized JSON content** 
Hey look Ma, it's JSON.....Let's view some rea-life JSON 

![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQt00IECBfWRzEFyW6D6dDnCvjuEt4MlTdF3WYi0IWg-_12ayXM&s)



In [0]:
response.json()  # this will show you serialized JSON content (See below).  to get a dictionary, you could take the str you received from .text and deserialize it (json.loads()) 
# much simplier to use .json(). the "type" of the return value .json() is a dictionary so you can access values i the object via the 'key'

**NOTE**: you can do a alot with status codes and message bodies, but in order to get more infomration, like METADATA about the response itself, you will need to review the response headers. 

##**API Response Headers**

The repsonse headers can provide lots of useful information such as content-type of the response payload, time-limit on how long to Cache the response etc..The View these headers, access (.headers)

In [0]:
response.headers

In [0]:
response.headers['content-Type']  # Accesing the key to content-type METADATA header.   NOT Case sensitive

In [0]:
response.headers['X-RateLimit-Remaining']

##**Maniupulating Query parameters (params)**

One common way to customize a GET request is to pass values thru Query String parameters in the URL. Todo this, you pass data to *params* 

query strings are useful for "parameterizing" GET requests. 

In [0]:
import requests

# search github's reposititories for requests 
response = requests.get('https://api.github.com/search/repositories', params=[('q', 'requests+language:python')], )
# return sever status
reponse.status_code

In [0]:
# you can pass values as Bytes
requests.get('https://api.github.com/search/repositories', params=b'q=requests+language:python')

##**Request Headers**

to customize headers,  you pass a **dictionary of HTTP headers** to GET() using the *'headers'* parameter. for exp, ypou can change your previous search request to highlight matching search terms in the results by specifying the "text-match media type in the *accept* header.

In [0]:
import requests

response = requests.get('https://api.github.com/search/repositories',
                        params={'q': 'requests+language:python'}, 
                        headers={'Accept': 'application/vnd.github.com.v3.text-match+json'},)

# View the new text matches array which provides informtion 
# about your search terms within the results
json_response = response.json()
repository = json_response['items'][0]
print(f'text matches: {repository["text_matches"]}') 

the accept header tells the server what content you application can handle. in thid case, since you're expecting the matching search terms to be highllighted , your using the header value "*application/vnd/github.v3.text-match+json*" which is a proprietart github accept header where the content is a special JSON format. 

before you learn ways to customize requests, Lets broden the horizon by exploring other HTTP methods. 

##**Other HTTP Methods**

Aside from GET, other popular HTTP methods include POST, PUT, DELETE, HEAD, PATCH and OPTIONS headers.  Requests provides a methodwith a similar signature to GET(), for each of these methods. 

Each function call makes a request to the HTTPbin service using the corresponding HTTP method. for each method, you can inspect each method as before. (*See Below*)

In [0]:
requests.post('https://httpbin.org/post', data={'key':'value'})
requests.put('https://httpbin.org/put', data={'key':'value'})
requests.delete('https://httpbin.org/delete') 
requests.head('https://httpbin.org/get')
requests.patch('https://httpbin.org/patch', data={'key':'value'})
requests.options('https://httpbin.org/get')

In [0]:
response = requests.head('https://httpbin.org/get')
response.headers['content-type']

In [0]:
response = requests.delete('https://httpbin.org/delete')
json_response = response.json()
json_response['args']

#**Message Body**

Headers, response bodies, status codes and more are returned in the response for each method. Now lets tke a closer look at POST, PUT and PATCH HTTP methods. According to the HTTP spec, POST/PUT and PATCH pass their data thru the body of an HTTP message instead of the URL. Using requests, you pass the payload to the corresponding ***data*** parameter. 


Data takes a dictionary, list of tuples or  file-like object. you will want to adapt the data you send in thr message body to the specific needs of the service your interacting with.  

Exp: if your requests "*content-Type*" is "*application/x-www-form-urlencoded*", you can send data as a dictionary 

In [0]:
requests.post('https://httpbin.org/post', data={'key':'value'})

In [0]:
requests.post('https://httpbin.org/post', data=[('key', 'value')])

If however, you need to send ***JSON data*** , you can use the json parameter. Whe youn pass JSON data via json , requests will serialize tour data and add correct "Content-Type" header for you. 

In [0]:
response = requests.post('https://httpbin.org/post', json={'key':'value'})
json_response = response.json()
json_response['data']

In [0]:
json_response['headers']['Content-Type']

You can see from the response that the server received your request data and headers as you sent,  requests also provides this infomration to you in a form of a PreparedRequests. 

#**Inspecting the Request**

When you make a request, the requests library prepares the request before actually sending to the destination server, preperation includes things such as "**header-validation**" and **serialization** of the data (*JSON Content)*. 

You can access the **PreparedRequests**  by accessing .request

In [0]:
response = requests.post('https://httpbin.org/post', json={'key':'value'})
response.request.headers['Content-type']

In [0]:
response.request.url

In [0]:
response.request.body

Inspecting the rquest gives tyou access to all kinds of information about the request being made:

*   Payload
*   URL
*   Headers
*   Authenication and more






#**API Authentication** (Basic Authentication Depricated  on Github)

Authenticaion helps the service understand who you are. Typically you provide credentials to the serverby passing data through the "Authorization Header" or custom header defined by th service. All the request function we studed so far have a paramter called "auth" which allows you to pass your credentials

One example of an API that requires authentication is Github's Authenticated user API. this endpoint provides infomation about the authenticated users profile. . To make the request to the Authenticated user API, pass your Github username and password in a Tuple to GET()

In [0]:
from requests.auth import AuthBase
from getpass import getpass
requests.get('https://api.github.com/user', auth='HTTPBasicAuth'('vlx300', getpass()))

#**SSL/TLS Security (Certificate Verification)**

Security is important. Requests uses SSL/TLS by default. Requests usees a package called cerifi to provide Certificate authorities.This lets request know wihch authorities it can trust. 

In [0]:
requests.get('https://api.github.com', verify=False)  # this will disable SSL and Provide a warning 

#**API Performance**

when using python requests in production, its important to consider performance implications . Features such as Timeout control, Sessions, Retry Limits, 

##**Timeouts**

when making inline requests to an external service, your system needs to wait for a reply before moving on. If application  waits too long, requests to the service could back up , or User experience could suffer, or backround jobs could hang.  By default, requests will wait indefinately for a connection so set some timeout value as a best prectice (Float or integer)

In [0]:
requests.get('https://api.github.com', verify=False, timeout=1) # Set a timeout value for the connection 

In [0]:
requests.get('https://api.github.com', timeout=2.5)

You can also pass a Tuple to timeout with the 1st element being the connecton-timeout value (*time allowed to establish connection*), and the 2nd value being the Read-timeout(*the time the connection will wait on a response*)

In [0]:
requests.get('https://api.github.com', timeout=(2,5)) # Tuple passed ot timeout. thjis request will be successful if the connection is established in 2seconds and data is received in 5 seconds #
# if the request timeos out, the function will raise a timeout exception. 

In [0]:
import requests
from requests.exceptions import Timeout

try: response = requests.get('https://api.github.com', timeout=1)
except TimeoutError:
  print("the request timeout")
else:
  print("The request did not time out")

#**Session Objects**

So far  We have been making high-level requests such as GET() and Post(). these are abstractions to what is really going on unde the hood. They hide the implementation details of the request.

Underneath these abstractions is a "Class" called Session.  If you need to have fine-tune controls over how the sessions are being made and managed.,  you may need to use a **Session instance directly**.  

Sessions are used to "persist" parameters accross connection requests. Exp, You want to use the "same authentication" accross muitiple requests

In [0]:
import requests
from getpass import getpass
# by using the context manager, you can ensure the resources used by the session
# will be released after use.
with requests.session() as session:
  session.auth = ('username', getpass())

  # instead of requests.get() you will use Session.get()
  response = session.get('https://api/github.com/user')

# you can inspect the repsonse as before 
print(response.headers)
print(response.json())

###**Primary performance optimization**

the primary optimization of "sessions" comes in the form of persistent connections. when your app makes a connection  to a server using a Session, it keeps the connection in the connection POOL so when the connection is needed again, *it uses the connection from the POOL rather than a new connection.* 

#**Max Retries**

when requests fails, you may want your application to retry the same request. By default request wont retry the connection. to implement this . you must implemnt a **Custom Transport Adapter**. 


Transport Adapters lets you define a set of configurations per service your interacting with. Exp.  you wan to retry https://api.github.com three times before finlly raising a "*Connection Error*", you would build a transport adapter, set its "*Max_Retires*" parameter and mount it to the existing session. 

In [0]:
import requests
from requests.adapters import HTTPAdapter
from requests.exceptions import ConnectionError

github_adapter = HTTPAdapter(max_retries=3)

session = requests.session()

# use gitbuh adapter for ALL requests to endpoints to start usint this URL
session.mount('https://api.github.com', github_adapter)

try: 
  session.get('https://api.github.com')
except ConnectionError as ce:
  print(ce)

when you mount the HTTPAdapter, github_adapter to session, session will adhere to its configuration for each request to https://api.github.com making sessions efficient and applications more resilient.